# 词频、互信息、信息熵发现中文新词

**新词发现**任务是中文自然语言处理的重要步骤。**新词**有“新”就有“旧”，属于一个相对个概念，在相对的领域（金融、医疗），在相对的时间（过去、现在）都存在新词。[文本挖掘](https://zh.wikipedia.org/wiki/文本挖掘)会先将文本[分词](https://zh.wikipedia.org/wiki/中文自动分词)，而通用分词器精度不过，通常需要添加**自定义字典**补足精度，所以发现新词并加入字典，成为文本挖掘的一个重要工作。

[**单词**](https://zh.wikipedia.org/wiki/單詞)的定义，来自维基百科的定义如下：
>在语言学中，**单词**（又称为词、词语、单字；英语对应用语为“word”）是能独立运用并含有语义内容或语用内容（即具有表面含义或实际含义）的最小单位。单词的集合称为词汇、术语，例如：所有中文单词统称为“中文词汇”，医学上专用的词统称为“医学术语”等。词典是为词语提供音韵、词义解释、例句、用法等等的工具书，有的词典只修录特殊领域的词汇。

单从语义角度，“苹果“的法语是"pomme"，而“土豆”的法语是"pomme de terre"，若按上面的定义，“土豆”是要被拆的面目全非，但"pomme de terre"是却是表达“土豆”这个语义的最小单位；在机构名中，这类问题出现的更频繁，"Paris 3"是"巴黎第三大学"的简称，如果"Paris"和"3"分别表示地名和数字，那这两个就无法表达“巴黎第三大学”的语义。而中文也有类似的例子，“北京大学”的”北京“和”大学“都可以作为一个最小单位来使用，分别表示”地方名“和“大学”，如果这样分词，那么就可以理解为“北京的大学”了，所以“北京大学”是一个表达语义的最小单位。前几年有部电影《夏洛特烦恼》，我们是要理解为“夏洛特 烦恼“还是”夏洛 特 烦恼“，这就是很经典的分词问题。

但是从语用角度，这些问题似乎能被解决，我们知道"pomme de terre"在日常生活中一般作为“土豆”而不是“土里的苹果”，在巴黎学习都知道“Paris 3”，就像我们提到“北京大学”特指那所著名的高等学府一样。看过电影《夏洛特烦恼》的观众很容易的就能区分这个标题应该看为“夏洛 特 烦恼”。

发现新词的方法，《[互联网时代的社会语言学：基于SNS的文本数据挖掘](http://www.matrix67.com/blog/archives/5044]) 》一文，里面提到的给每一个文本串计算**文本片段**的**凝固程度**和文本串对外的使用**自由度**，通过设定阈值来将文本串分类为词和非词两类。原文给了十分通俗易懂的例子来解释凝固度和自动度。这里放上计算方法。这个方法还有许多地方需要优化，在之后的实践中慢慢调整了。

## 文本片段

**文本片段**，最常用的方法就是[n元语法(ngram)](https://zh.wikipedia.org/wiki/N元语法)，将分本分成多个n长度的文本片段。数据结构，这里采用Trie树的方案，这个方案是简单容易实现，而且用Python的字典做Hash索引实现起来也很优美，唯独的一个问题是所有的数据都存在内存中，这会使得内存占用量非常大，如果要把这个工程化使用，还需要采用其他方案，比如硬盘检索。
<a href="https://upload.wikimedia.org/wikipedia/commons/b/be/Trie_example.svg
" target="_blank"><img src="https://upload.wikimedia.org/wikipedia/commons/b/be/Trie_example.svg" 
alt="IMAGE ALT TEXT HERE" width="240" height="180" border="10" /></a>

In [ ]:
class TrieNode(object):
    def __init__(self, 
                 frequence=0, 
                 children_frequence=0, 
                 parent=None):

        self.parent = parent
        self.frequence = frequence
        self.children = {} 
        self.children_frequence = children_frequence

    def insert(self, char):
        self.children_frequence += 1
        self.children[char] = self.children.get(char, TrieNode(parent=self))
        self.children[char].frequence += 1
        return  self.children[char]
        
    def fetch(self, char):
        return self.children[char]
    
class TrieTree(object):
    def __init__(self, size=6):
        self._root = TrieNode()
        self.size = size
        
    def get_root(self):
        return self._root
    
    def insert(self, chunk):
        node = self._root
        for char in chunk:
            node = node.insert(char)
        if len(chunk) < self.size:
            # add symbol "EOS" at end of line trunck
            node.insert("EOS")

    def fetch(self, chunk):
        node = self._root
        for char in chunk:
            node = node.fetch(char)
        return node

Trie树的结构上，我添加了几个参数，parent，frequence，children_frequence，他们分别是：
- parent，当前节点的父节点，如果是“树根”的时候，这个父节点为空；
- frequence，当前节点出现的频次，在Trie树上，也可以表示某个文本片段的频次，比如"中国"，“国”这个节点的frequence是100的时候，“中国”俩字也出现了100次。这个可以作为最后的词频过滤用。
- children_frequence，当前接点下有子节点的"frequence"的总和。比如在刚才的例子上加上“中间”出现了99次，那么“中”这个节点的children_frequence的值是199次。
这样的构造让第二部分的计算更加方面。

这个任务中需要构建两棵Trie树，表示正向和反向两个字符片段集。

## 自由度

**自由度**，使用信息论中的[信息熵](https://zh.wikipedia.org/wiki/熵_(信息论))构建文本片段左右熵，公式[1]。熵越大，表示该片段和左右邻字符相互关系的不稳定性越高，那么越有可能作为独立的片段使用。公式[1]第一个等号后面的I(x)表示x的自信息。
\begin{align} 
 H(X) = \sum_{i} {\mathrm{P}(x_i)\,\mathrm{I}(x_i)} = -\sum_{i} {\mathrm{P}(x_i) \log \mathrm{P}(x_i)} [1]
\end{align} 

In [ ]:
def calc_entropy(chunks, ngram):
    """计算信息熵
    Args:
        chunks，是所有数据的文本片段
        ngram，是Trie树
    Return:
        word2entropy，返回一个包含每个chunk和对应信息熵的字典。
    """
    def entropy(sample, total):
        """Entropy"""
        s = float(sample)
        t = float(total)
        result = - s/t * math.log(s/t)
        return result

    def parse(chunk, ngram):
        node = ngram.fetch(chunk)
        total = node.children_frequence
        return sum([entropy(sub_node.frequence, 
                           total) for sub_node in node.children.values()])

    word2entropy = {}
    for chunk in chunks:
        word2entropy[chunk] = parse(chunk, ngram)   
    return word2entropy

## 凝固度

**凝固度**，用信息论中的**互信息**表示，公式[2]。在概率论中，如果x跟y不相关，则p(x,y)=p(x)p(y)。二者相关性越大，则p(x,y)就相比于p(x)p(y)越大。用后面的式子可能更好理解，在y出现的情况下x出现的条件概率p(x|y)除以x本身出现的概率p(x)，自然就表示x跟y的相关程度。 
\begin{align} 
I(x;y) = \log\frac{p(x,y)}{p(x)p(y)} = \log\frac{p(x|y)}{p(x)} = \log\frac{p(y|x)}{p(y)} [2]
\end{align}

这里比较容易产生一个概念的混淆，维基百科将式[2]定义为[点互信息](https://en.wikipedia.org/wiki/Pointwise_mutual_information)，[互信息](https://zh.wikipedia.org/wiki/互信息)的定义如下：
\begin{align} 
I(X;Y) = \sum_{y \in Y} \sum_{x \in X} 
                 p(x,y) \log{ \left(\frac{p(x,y)}{p(x)\,p(y)}
                              \right) }\ [3]
\end{align}
在傅祖芸编著的《信息论——基础理论与应用（第4版）》的绪论中，把式[2]定义为互信息，而式[3]定义为平均互信息，就像信息熵指的是**平均自信息**。

In [ ]:
    def calc_mutualinfo(self, chunks, ngram):
        """计算互信息
        Args:
            chunks，是所有数据的文本片段
            ngram，是Trie树
        Return:
            word2mutualinfo，返回一个包含每个chunk和对应互信息的字典。
        """
        def parse(chunk, root):
            sub_node_y_x = ngram.fetch(chunk)
            node = sub_node_y_x.parent
            sub_node_y = root.children[chunk[-1]]
            
            # 这里采用互信息log(p(y|x)/p(y))的计算方法
            prob_y_x = float(sub_node_y_x.frequence) / node.children_frequence
            prob_y = float(sub_node_y.frequence) / root.children_frequence
            mutualinfo = math.log(prob_y_x / prob_y)
            return mutualinfo, sub_node_y_x.frequence
        
        word2mutualinfo = {}  
        root = ngram.get_root()
        for chunk in chunks:
            word2mutualinfo[chunk] = parse(chunk, root)
        return word2mutualinfo

## 过滤

最终计算得出互信息、信息熵，甚至也统计了词频，最后一步就是根据阈值对词进行过滤。

In [ ]:
def _fetch_final(fw_entropy，
                 bw_entropy,
                 fw_mi,
                 bw_mi
                 entropy_threshold=0.8,
                 mutualinfo_threshold=7,
                 freq_threshold=10):
        final = {}
        for k, v in fw_entropy.items():
            last_node = self.fw_ngram
            if k[::-1] in bw_mi and k in fw_mi:
                mi_min = min(fw_mi[k][0], bw_mi[k[::-1]][0])
                word_prob = min(fw_mi[k][1], bw_mi[k[::-1]][1])
                if mi_min < mutualinfo_threshold:
                    continue
            else:
                continue
            if word_prob < freq_threshold:
                 continue
            if k[::-1] in bw_entropy:
                en_min = min(v, bw_entropy[k[::-1]])
                if en_min < entropy_threshold:
                    continue
            else:
                continue
            final[k] = (word_prob, mi_min, en_min)
        return final

## 结果

最终，通过这个方法对这次十九大的开幕发言做的一个词汇发现，ngram的n=10，结果按词频排序输出，可以发现这次十九大谈了许多内容，不一一说了。这个结果还存在不少问题，比如“二〇”，这在阈值的设置上还不够准确，可以尝试使用机器学习的方法来获取阈值。

经济|70 改革|69 我们|64 必须|61 领导|60 完善|57 历史|44 不断|43 群众|43 教育|43 战略|42 思想|40 世界|39 问题|37 提高|37 组织|36 监督|35 加快|35 依法|34 精神|33 团结|33 复兴|32 保障|31 奋斗|30 根本|29 环境|29 军队|29 开放|27 服务|27 理论|26 干部|26 创造|26 基础|25 意识|25 维护|25 协商|24 解决|24 贯彻|23 斗争|23 目标|21 统筹|20 始终|19 方式|19 水平|19 科学|19 利益|19 市场|19 基层|19 积极|18 马克思|18 反对|18 道路|18 自然|18 增长|17 科技|17 稳定|17 原则|17 两岸|17 取得|16 质量|16 农村|16 矛盾|16 协调|15 巩固|15 收入|15 绿色|15 自觉|15 方针|15 纪律|15 长期|15 保证|15 同胞|15 命运|14 美好生活|14 五年|14 传统|14 繁荣|14 没有|14 使命|13 广泛|13 日益|13 价值|13 健康|13 资源|13 参与|13 突出|13 腐败|13 充分|13 梦想|13 任何|13 二〇|13 代表|12 阶段|12 深刻|12 布局|12 区域|12 贸易|12 核心|12 城乡|12 生态文明|12 工程|12 任务|12 地区|12 责任|12 认识|12 胜利|11 贡献|11 覆盖|11 生态环境|11 具有|11 面临|11 各种|11 培育|11 企业|11 继续|10 团结带领|10 提升|10 明显|10 弘扬|10 脱贫|10 贫困|10 标准|10 注重|10 基本实现|10 培养|10 青年|10 这个|9 复杂|9 改革开放|9 显著|9 成果|9 各方面|9 文艺|9 牢牢|9 台湾|9 外交|9 形式|9 担当|9 风险|9 提供|9 综合|9 树立|9 网络|9 劳动|9 旗帜|8 夺取|8 激励|8 奋斗目标|8 形势|8 城镇|8 司法|8 意识形态|8 地位|8 价值观|8 宪法|8 交流|8 分裂|8 路线|8 学习|8 考验|8 平衡|8 特别|8 自己|8 自身|8 富裕|8 美丽|8 百年|8 更加自觉地|8 各项|8 维护国家|8 技术|8 队伍|8 自由|8 关键|7 我国经济|7 供给|7 投资|7 相互|7 监察|7 指导|7 优秀|7 影响|7 医疗|7 赋予|7 条件|7 权威|7 反腐败|7 分配|7 伟大梦想|7 伟大斗争|7 宗旨|7 信息|7 渠道|7 道德|7 污染|7 优先|7 金融|7 服务体系|7 职责|7 十八|6 初心|6 幸福|6 永远|6 观念|6 突破|6 宗教|6 文化自信|6 卫生|6 生态文明建设|6 节约|6 两岸关系|6 良好|6 理想信念|6 许多|6 未来|6 什么|6 规律|6 优势|6 规范|6 不动摇|6 鼓励|6 能够|6 竞争|6 消费|6 经营|6 防止|6 现代化建设|5 转变|5 规划|5 千多|5 破除|5 统一战线|5 鲜明|5 精神文明|5 不断提高|5 互联|5 为中心|5 状况|5 准备|5 港澳|5 九二|5 推动两岸|5 层次|5 营造|5 威胁|5 方针政策|5 凝聚|5 不平衡不充分|5 差距|5 养老|5 特别行政区|5 台湾同胞|5 希望|5 智慧|5 我国社会主要矛盾|5 艰苦|5 深刻认识|5 武装|5 充满|5 敢于|5 反对一切|5 防范|5 公有制经济|5 素质|5 可持续|5 到二〇|5 二〇二〇年|5 按照|5 经济体系|5 知识|5 研究|5 底线|5 事情|5 牢记|4 不懈奋斗|4 懈怠|4 处于|4 前景|4 历史性变|4 世界经济|4 部署|4 总体布局|4 质量和效益|4 第二|4 逐步|4 举措|4 试点|4 理论创新|4 互联网|4 以人民为中心|4 西部|4 医疗卫生|4 功能|4 修复|4 繁荣稳定|4 世界和平|4 尊崇|4 突出问题|4 巡视|4 容忍|4 反腐败斗争|4 压倒性|4 已经|4 近代以|4 独立|4 物质|4 错误|4 损害|4 脱离|4 除一切|4 始终成为|4 紧密|4 本世纪中叶|4 指挥|4 军队建设|4 长期坚持|4 丰富|4 前途|4 毫不动摇|4 非公有制经济|4 配置|4 推动经济|4 相结合|4 保障和改善民生|4 共建共|4 必须全面|4 两岸同胞|4 秩序|4 振兴|4 公共服务|4 低碳|4 技术创新|4 土地|4 边疆|4 贫困地区|4 根本利益|4 通过|4 履行|4 超越|4 多样|4 交往|4 传播|4 空间|4 讴歌|4 传承|4 保险制度|4 扶贫|4 清洁|4 危险|4 基层党组织|4 召开|3 姿态|3 朝着|3 严峻|3 常态|3 推进依法|3 经济建设|3 经济增长|3 百分之|3 供给侧结构性改革|3 蓬勃|3 基础设施|3 达到|3 多万|3 创新驱动|3 创新型国家|3 改革取得|3 框架|3 迈出|3 爱国统一战线|3 权力运行|3 监督体系|3 意识形态工作|3 意识形态领域|3 文化事业|3 文化产业|3 获得感|3 攻坚战|3 贫困人口|3 教育事业|3 生态环境保护|3 节约资源|3 生态保护|3 环境治理|3 新形势下|3 恢复|3 优良|3 军事斗争准备|3 牢牢掌握|3 两岸经济|3 多层次|3 维护党中央权威|3 风气|3 零容忍|3 构筑|3 了许多|3 重大风险|3 困难|3 艰巨|3 收入分配|3 措施|3 那时|3 近代以来|3 选择|3 舞台|3 为人类作出|3 总体上|3 物质文|3 提出了|3 因素|3 牢牢把握|3 过程|3 五千多年|3 负起|3 复兴的历史|3 团结带领人民进行|3 时代潮流|3 顺应|3 无论|3 不渝|3 付出|3 攻克|3 我国社会主义制度|3 脱离群众|3 改革创新|3 破坏|3 各种风|3 必须始终|3 纯洁性|3 侵蚀|3 永葆|3 封闭|3 伟大实践|3 总目标|3 围绕|3 探索|3 打胜仗|3 世界一流|3 总要求|3 继承|3 东西|3 牢固树立|3 共建共享|3 必须全面贯彻|3 聚焦|3 军民融合|3 奉行|3 包容|3 遵循|3 摆在|3 纠正|3 及其|3 新征程|3 再奋斗|3 乡村|3 到本世纪中叶|3 二〇三五年|3 缩小|3 市场经济|3 产业体系|3 宏观|3 调控|3 绿色低碳|3 创新体系|3 农业农村|3 承包|3 约束|3 直接|3 便利|3 自由贸易|3 逻辑|3 法治保障|3 领导干部|3 接受|3 担负|3 宪法法律|3 专门|3 配套|3 级党组织|3 带头|3 市县|3 多样性|3 紧紧|3 适应|3 他们|3 手段|3 综合治理|3 家庭|3 宣传|3 勤劳|3 调节|3 养老保险|3 保障制度|3 打赢|3 到二〇二〇年|3 预防|3 防控|3 恐怖|3 循环|3 生态系统|3 长期繁荣稳定|3 对话|3 秉持|3 维护国家主权|3 放弃|3 超越文明|3 组织建设|3 忠诚|3 基层组织|3 路线方针政策|3 权限|3 震慑|3 初心和使命|2 是为中国人民谋幸福|2 朝着实现|2 仍处于|2 机遇|2 过去|2 对世界经济|2 冲突|2 外部环境|2 顺利|2 事业全面|2 高速增长|2 万亿元|2 稳居世界|2 世界经济增长|2 超过|2 经济结构|2 桥梁|2 等基础设施|2 粮食|2 年均|2 二个百|2 千多万|2 农业转移人口|2 重大科技|2 科技成果|2 开放型经济|2 对外投资|2 储备|2 推进全面|2 纵深|2 增强改革|2 更加广泛|2 宗教工作|2 公共文化服务|2 文化事业和文化产业|2 脱贫攻坚战|2 下降|2 居民收入|2 住房|2 加快形成|2 保护和修复|2 森林|2 为全球生态|2 引领者|2 着眼|2 光荣|2 有效治理|2 军队组织|2 救灾|2 港澳台|2 宪法和基本法|2 外交布局|2 组织领导|2 我国国际|2 塑造|2 世界和平与|2 好干部标准|2 群众反映|2 省级|2 不能腐的笼子|2 反腐败斗争压倒性|2 压倒性态势|2 出一系列|2 而没有|2 于面对|2 气象|2 性锻|2 焕发出|2 清醒|2 看到|2 还存在|2 面临不少|2 突出问题尚未|2 收入分配差距|2 重大政策|2 特别行政区同胞|2 侨胞|2 衷心|2 号召|2 姿态屹立于世界|2 经磨难|2 迎来了|2 二十一世纪|2 途径|2 那些|2 而且|2 并将|2 基本国情|2 基本路线|2 谋求|2 文明历史|2 成为世界|2 山河|2 不屈不挠|2 谱写|2 找到|2 牺牲|2 了一个又一个|2 更为艰|2 抵御|2 行为都|2 投身|2 反对一切分裂|2 各种风险|2 将继续|2 先锋|2 先进性和纯洁性|2 理论和实践|2 贯通|2 无愧|2 包括|2 分析|2 辩证|2 时代条件|2 目标是建设|2 内涵|2 各项工作|2 南北|2 前途命运|2 为人民服务|2 充分发挥|2 资源配置|2 素养|2 文化创造|2 福祉|2 新时代党的强军思想|2 推动两岸同胞共同|2 始终不渝|2 互惠|2 文明交流|2 抓住|2 少数|2 严肃|2 也没有|2 这两个|2 后再|2 检验|2 交汇|2 第一个|2 个百年奋斗目标|2 第二个|2 年到本世纪中叶|2 物质文明|2 生产率|2 构建市场|2 宏观调控|2 增强我国经济|2 着力点|2 先进制造业|2 深度融合|2 瞄准|2 若干|2 前沿|2 项目|2 颠覆|2 战略科技|2 造就一大批|2 科技人才|2 土地承包|2 产权制度|2 财产权|2 现代农业|2 规模|2 革命老区|2 率先|2 北京|2 陆海|2 资产管理|2 调整|2 有效防止|2 流失|2 市场准入|2 负面清单|2 除妨碍|2 放宽|2 货币|2 经济政策|2 投融资|2 均衡|2 支柱|2 落后|2 共商共建共享|2 海内外|2 待遇|2 对外开放|2 稳妥|2 程序|2 各级领导|2 担负起|2 日常|2 贯穿|2 重大方针政策|2 决策部署|2 让人民群众|2 绝不允许|2 违法|2 理论武装|2 构建中国特色|2 舆论|2 教育引导|2 习惯|2 时代要求|2 永久|2 道德建设|2 信仰|2 觉悟|2 品德|2 素质教|2 培训|2 高等教育|2 接受高|2 学习型|2 提供全方位|2 多渠道|2 基本养老保险|2 城乡居民基本|2 基本医疗|2 妇女|2 儿童|2 残疾|2 是用来|2 负总责|2 周期|2 医药|2 医疗卫生服务体系|2 安全战略|2 老龄|2 负责|2 打击|2 财富|2 生产和消费|2 处置|2 污染排放|2 评价|2 生态系统保护|2 耕地|2 有自然资源资产|2 联合作战|2 推进军事|2 到二〇三五年|2 深化国防|2 职人员|2 打仗|2 做好各|2 安全领域|2 粤港澳|2 共担民族|2 任何形式|2 外交政策|2 友好|2 非传统安全威胁|2 不能因|2 而放弃|2 没有哪个国家能够|2 亲诚|2 邻为|2 周边|2 持正确义利观|2 畅通|2 把党建设|2 永远在路上|2 痛恨|2 纯洁性的因素|2 推进反腐败|2 廉洁|2 锻炼|2 共产党人的精神|2 干部队伍|2 基层一线|2 选拔|2 认真|2 为他们|2 天下|2 才的良|2 批评|2 监督执纪|2 压倒性胜利|2 巡察|2 十三亿多|2 锐意进取|2 群众工作|2 青年一代|2 维护世界和平|2

## 代码下载地址

git clone https://github.com/KunFly/new-word-discovery.git